In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.utils import to_categorical 

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

df = pd.read_csv('datamahasiswa2015.csv')
df.head()

,No,NIM,OOP,RPB,RPL,PI,APSI,WEB,Statistik,Matdis,...,Desjar,KSI,Pilihan 1,Peminatan,MP1,MP2,MP3,Status,Tanggal Lulus,Keterangan
0,1,1202150001,2.0,3.5,0.0,3.5,3.0,3.0,2.0,2.0,...,3.5,2.0,TECHNO,TECHNO,3.5,3.0,3.0,GRADUATED,30/08/19,TEPAT WAKTU
1,2,1202150002,3.0,3.5,3.5,4.0,3.0,3.0,3.5,3.5,...,3.5,3.5,ERP,ERP,4.0,4.0,3.5,GRADUATED,10/07/19,TEPAT WAKTU
2,3,1202150003,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,OUT,OUT,0.0,0.0,0.0,RESIGN,OUT,TIDAK TEPAT WAKTU
3,4,1202150004,3.5,4.0,3.5,4.0,3.5,4.0,3.5,3.5,...,4.0,4.0,EAD,EAD,3.5,4.0,2.0,GRADUATED,05/07/19,TEPAT WAKTU
4,5,1202150005,2.0,2.5,2.0,1.0,3.5,1.0,2.0,1.0,...,2.0,1.0,OUT,OUT,0.0,0.0,0.0,STUDENT,OUT,TIDAK TEPAT WAKTU


In [3]:
new_class = [0] * len(df['Keterangan'])
for index, row in df.iterrows() :
    if row['Keterangan'] == "TEPAT WAKTU" :
        new_class[index] = 1
    else :
        new_class[index] = 0

In [5]:
df["ket_status"] = new_class
df.head()

,No,NIM,OOP,RPB,RPL,PI,APSI,WEB,Statistik,Matdis,...,KSI,Pilihan 1,Peminatan,MP1,MP2,MP3,Status,Tanggal Lulus,Keterangan,ket_status
0,1,1202150001,2.0,3.5,0.0,3.5,3.0,3.0,2.0,2.0,...,2.0,TECHNO,TECHNO,3.5,3.0,3.0,GRADUATED,30/08/19,TEPAT WAKTU,1
1,2,1202150002,3.0,3.5,3.5,4.0,3.0,3.0,3.5,3.5,...,3.5,ERP,ERP,4.0,4.0,3.5,GRADUATED,10/07/19,TEPAT WAKTU,1
2,3,1202150003,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,OUT,OUT,0.0,0.0,0.0,RESIGN,OUT,TIDAK TEPAT WAKTU,0
3,4,1202150004,3.5,4.0,3.5,4.0,3.5,4.0,3.5,3.5,...,4.0,EAD,EAD,3.5,4.0,2.0,GRADUATED,05/07/19,TEPAT WAKTU,1
4,5,1202150005,2.0,2.5,2.0,1.0,3.5,1.0,2.0,1.0,...,1.0,OUT,OUT,0.0,0.0,0.0,STUDENT,OUT,TIDAK TEPAT WAKTU,0


In [7]:
mainData = df.drop(['Status','Keterangan','Pilihan 1','Peminatan','Tanggal Lulus','No','NIM','MP1','MP2','MP3','RPL'],axis=1)
mainData.rename(columns={'KSI':'Manprosi'}, inplace=True)
mainData.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,ket_status
0,2.0,3.5,3.5,3.0,3.0,2.0,2.0,3.0,2.0,3.5,3.5,3.0,3.5,2.0,2.0,2.5,3.5,3.5,2.0,1
1,3.0,3.5,4.0,3.0,3.0,3.5,3.5,3.5,3.0,3.5,4.0,4.0,3.0,3.5,2.5,3.5,3.5,3.5,3.5,1
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
3,3.5,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0,3.0,4.0,4.0,3.0,3.5,3.5,3.5,3.5,4.0,4.0,1
4,2.0,2.5,1.0,3.5,1.0,2.0,1.0,2.5,0.0,2.5,4.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,0


In [8]:
target = ["ket_status"]
predictors = list(set(list(mainData.columns))-set(target))
mainData[predictors] = mainData[predictors]/mainData[predictors].max()
mainData[predictors].head()

,WEB,APSI,Manprosi,SCM,Desjar,Alpro,PI,OOP,Matdis,PO,Statistik,MSDM,SE,Basdat,Sisop,Manjarkom,EA,Strukdat,RPB
0,0.75,0.750,0.500,0.75,0.875,0.750,0.875,0.500,0.500,0.875,0.500,0.875,0.875,0.500,0.625,0.500,0.875,0.50,0.875
1,0.75,0.750,0.875,1.00,0.875,0.875,1.000,0.750,0.875,1.000,0.875,0.875,0.875,0.875,0.875,0.625,0.750,0.75,0.875
2,0.00,0.250,0.000,0.00,0.250,0.000,0.000,0.250,0.000,0.000,0.000,0.000,0.500,0.000,0.000,0.000,0.000,0.50,0.000
3,1.00,0.875,1.000,1.00,1.000,0.875,1.000,0.875,0.875,1.000,0.875,0.875,0.750,0.875,0.875,0.875,0.750,1.00,1.000
4,0.25,0.875,0.250,0.50,0.500,0.625,0.250,0.500,0.250,1.000,0.500,0.250,0.625,0.250,0.500,0.500,0.500,0.00,0.625


In [9]:
X = mainData[target]
y = mainData[predictors]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape)
print(y_train.shape)
print(X_test.head())
print(y_test.shape)

(259, 1)
(259, 19)
     ket_status
263           1
314           1
296           0
26            1
185           1
(111, 19)


In [10]:
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

X_trains = to_categorical(X_train)
X_tests = to_categorical(X_test)

print(X_trains.shape)

(259, 2)


In [11]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=19))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [12]:
model.fit(y_train, X_trains, epochs=200)

Epoch 1/200
259/259 [==============================] - 0s 696us/step - loss: 0.5757 - accuracy: 0.7066
Epoch 2/200
259/259 [==============================] - 0s 73us/step - loss: 0.4807 - accuracy: 0.7375
Epoch 3/200
259/259 [==============================] - 0s 62us/step - loss: 0.4633 - accuracy: 0.7761
Epoch 4/200
259/259 [==============================] - 0s 58us/step - loss: 0.4418 - accuracy: 0.8069
Epoch 5/200
259/259 [==============================] - 0s 58us/step - loss: 0.4125 - accuracy: 0.8224
Epoch 6/200
259/259 [==============================] - 0s 58us/step - loss: 0.3936 - accuracy: 0.8263
Epoch 7/200
259/259 [==============================] - 0s 62us/step - loss: 0.3633 - accuracy: 0.8571
Epoch 8/200
259/259 [==============================] - 0s 54us/step - loss: 0.3219 - accuracy: 0.8649
Epoch 9/200
259/259 [==============================] - 0s 62us/step - loss: 0.3164 - accuracy: 0.8571
Epoch 10/200
259/259 [==============================] - 0s 58us/step - loss: 0.28

In [36]:
testingData = df.drop(['Status','Keterangan','Pilihan 1','Peminatan','Tanggal Lulus','No','MP1','MP2','MP3','RPL','ket_status'],axis=1)
testingrename = testingData.rename(columns={'KSI':'Manprosi'})
testingrename.head()

,NIM,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi
0,1202150001,2.0,3.5,3.5,3.0,3.0,2.0,2.0,3.0,2.0,3.5,3.5,3.0,3.5,2.0,2.0,2.5,3.5,3.5,2.0
1,1202150002,3.0,3.5,4.0,3.0,3.0,3.5,3.5,3.5,3.0,3.5,4.0,4.0,3.0,3.5,2.5,3.5,3.5,3.5,3.5
2,1202150003,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1202150004,3.5,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0,3.0,4.0,4.0,3.0,3.5,3.5,3.5,3.5,4.0,4.0
4,1202150005,2.0,2.5,1.0,3.5,1.0,2.0,1.0,2.5,0.0,2.5,4.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0


In [38]:
dataTestingFilter = testingrename[predictors]
dataTestingHasil = model.predict_classes(dataTestingFilter)

testingrename["Lulus Tepat Waktu"] = dataTestingHasil
testingrename.rename(columns={'nim':'NIM'}, inplace=True)
testingrename.head()

,NIM,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,...,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu
0,1202150001,2.0,3.5,3.5,3.0,3.0,2.0,2.0,3.0,2.0,...,3.5,3.0,3.5,2.0,2.0,2.5,3.5,3.5,2.0,1
1,1202150002,3.0,3.5,4.0,3.0,3.0,3.5,3.5,3.5,3.0,...,4.0,4.0,3.0,3.5,2.5,3.5,3.5,3.5,3.5,1
2,1202150003,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
3,1202150004,3.5,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0,...,4.0,4.0,3.0,3.5,3.5,3.5,3.5,4.0,4.0,1
4,1202150005,2.0,2.5,1.0,3.5,1.0,2.0,1.0,2.5,0.0,...,4.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,0
